In [ ]:
from pyspark import SparkContext
sc = SparkContext( 'local', 'mywordcount')
textFile = sc.textFile("file:///opt/text/python/pyspark/data.txt")
wordCount = textFile.flatMap(lambda line: line.split(" ")).map(lambda word: (word,1)).reduceByKey(lambda a, b : a + b)
wordCount.collect()
wordCount.foreach(print)


# RDD综合应用 

In [ ]:
import random as rd
for i in range(10):
    num=rd.random()*100
    str1=str(i)+ " "+str(int(num))
    print(str1)

### RDD操作计算平均年龄

由年龄文件生成人rdd1

In [ ]:
from pyspark import SparkContext
sc = SparkContext( 'local', 'mywordcount')
rdd1=sc.textFile("age.txt")
rdd1.collect()

RDD转换（年龄，1）形式的RDD

In [ ]:
rdd2 = rdd1.map(lambda line : line.split(" ")).map(lambda x:(int(x[1]),1))
rdd2.collect()

In [ ]:
rdd2.keys().reduce(lambda x,y : x+y)/rdd2.values().count()

In [ ]:
rdd = sc.parallelize([("spark",2),("hadoop",6),("hadoop",4),("spark",6)])
rdd.mapValues(lambda x : (x,1)).reduceByKey(lambda x,y : (x[0]+y[0],x[1] + y[1])).mapValues(lambda x : (x[0] / x[1])).collect()


In [ ]:
rdd=sc.parallelize(["b","a","c"])
rdd.persist()

In [ ]:
rdd.is_cached

In [ ]:
rdd.count()

In [ ]:
rdd.unpersist()
rdd.is_cached

In [ ]:
import pandas as pd
print(pd.__version__)

# 实验四---SparkDataFram 

In [2]:

from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("rddDataFrame").master("local[*]").getOrCreate()
#读取数据
book=spark.read.text('file:///opt/text/python/pyspark/book.txt')
book.take(5)

[Row(value='序号,书名,评分,价格,出版社,url'),
 Row(value='5173,動力取向精神醫學--臨床應用與實務,10.0 ,1200元,心灵工坊,https://book.douban.com/subject/6053667/'),
 Row(value='9929,水彩绘森活,10.0 ,29.8,人民邮电出版社,https://book.douban.com/subject/26115807/'),
 Row(value='10124,殷周金文集成(修订增补本共8册)(精),10.0 ,2400.00元,中华书局,https://book.douban.com/subject/2235855/'),
 Row(value='16628,纸雕游戏大书,10.0 ,99.00元,重庆出版集团,https://book.douban.com/subject/26673804/')]

In [3]:
type(book)

pyspark.sql.dataframe.DataFrame

In [4]:
book.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
book_rdd=book.rdd.map(lambda x:x[0].split(",")).map(lambda x:Row(id=x[0],name=x[1],rating=x[2],price=x[3],publish=x[4],rul=x[5]))
book_rdd.take(5)

[Row(id='序号', name='书名', rating='评分', price='价格', publish='出版社', rul='url'),
 Row(id='5173', name='動力取向精神醫學--臨床應用與實務', rating='10.0 ', price='1200元', publish='心灵工坊', rul='https://book.douban.com/subject/6053667/'),
 Row(id='9929', name='水彩绘森活', rating='10.0 ', price='29.8', publish='人民邮电出版社', rul='https://book.douban.com/subject/26115807/'),
 Row(id='10124', name='殷周金文集成(修订增补本共8册)(精)', rating='10.0 ', price='2400.00元', publish='中华书局', rul='https://book.douban.com/subject/2235855/'),
 Row(id='16628', name='纸雕游戏大书', rating='10.0 ', price='99.00元', publish='重庆出版集团', rul='https://book.douban.com/subject/26673804/')]

In [6]:
book_df=spark.createDataFrame(book_rdd)
book_df.show(5)

+-----+---------------------------------+------+---------+--------------+--------------------+
|   id|                             name|rating|    price|       publish|                 rul|
+-----+---------------------------------+------+---------+--------------+--------------------+
| 序号|                             书名|  评分|     价格|        出版社|                 url|
| 5173| 動力取向精神醫學--臨床應用與實務| 10.0 |   1200元|      心灵工坊|https://book.doub...|
| 9929|                       水彩绘森活| 10.0 |     29.8|人民邮电出版社|https://book.doub...|
|10124|殷周金文集成(修订增补本共8册)(精)| 10.0 |2400.00元|      中华书局|https://book.doub...|
|16628|                     纸雕游戏大书| 10.0 |  99.00元|  重庆出版集团|https://book.doub...|
+-----+---------------------------------+------+---------+--------------+--------------------+
only showing top 5 rows



In [7]:
book_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- price: string (nullable = true)
 |-- publish: string (nullable = true)
 |-- rul: string (nullable = true)



In [8]:
#指定列输出
book_df.select('id','name').show()

+-----+-----------------------------------+
|   id|                               name|
+-----+-----------------------------------+
| 序号|                               书名|
| 5173|   動力取向精神醫學--臨床應用與實務|
| 9929|                         水彩绘森活|
|10124|  殷周金文集成(修订增补本共8册)(精)|
|16628|                       纸雕游戏大书|
|19103|                       Michelangelo|
|20063|                  一支笔的快乐涂鸦2|
|32781|                         亲亲宝贝装|
|32879|                     Photoshop7解像|
|45687|                   戚蓼生序本石头记|
|52504|                      宇宙兄弟（7）|
|52505|                      宇宙兄弟（8）|
|  573|            TCP\IP详解（卷1英文版）|
|  589|计算机程序设计艺术卷1：基本算法(...|
| 5522|         微积分和数学分析引论-第1卷|
| 5547|               PrinciplesofNeura...|
| 7443|           奈特人体神经解剖彩色图谱|
| 8703|                 数学、科学和认识论|
| 9924|                       零基础学素描|
| 9926|     黑白花意3：300例超写实的花之绘|
+-----+-----------------------------------+
only showing top 20 rows



In [9]:
#查看方法和属性
dir(book_df)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collect_as_arrow',
 '_jcols',
 '_jdf',
 '_jmap',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_sort_cols',
 '_support_repr_html',
 '_to_corrected_pandas_type',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'drop_duplicates',
 'dropna',
 'dtypes',
 'exceptAll',
 'explain',
 'fillna',
 'fi

In [10]:
book_df.select(book_df['id'],book_df['name'],book_df['rating']-1).show()

+-----+-----------------------------------+------------+
|   id|                               name|(rating - 1)|
+-----+-----------------------------------+------------+
| 序号|                               书名|        null|
| 5173|   動力取向精神醫學--臨床應用與實務|         9.0|
| 9929|                         水彩绘森活|         9.0|
|10124|  殷周金文集成(修订增补本共8册)(精)|         9.0|
|16628|                       纸雕游戏大书|         9.0|
|19103|                       Michelangelo|         9.0|
|20063|                  一支笔的快乐涂鸦2|         9.0|
|32781|                         亲亲宝贝装|         9.0|
|32879|                     Photoshop7解像|         9.0|
|45687|                   戚蓼生序本石头记|         9.0|
|52504|                      宇宙兄弟（7）|         9.0|
|52505|                      宇宙兄弟（8）|         9.0|
|  573|            TCP\IP详解（卷1英文版）|         8.9|
|  589|计算机程序设计艺术卷1：基本算法(...|         8.9|
| 5522|         微积分和数学分析引论-第1卷|         8.9|
| 5547|               PrinciplesofNeura...|         8.9|
| 7443|           奈特人体神经解剖彩色图谱|

In [11]:
import pandas as pd
book_pds=book_df.groupBy('publish').count()
print(book_pds.show())

+------------------------------------+-----+
|                             publish|count|
+------------------------------------+-----+
|                         SearchPress|    3|
|                McGraw-HillScienc...|    6|
|                            上海教育|    5|
|                ScholasticPaperbacks|    2|
|                                 DAW|    1|
|                         PanBooksLtd|    1|
|                  奇幻基地出版事業部|    1|
|                               Ember|    1|
|                         Allyn&Bacon|    1|
|                     WalkerArtCenter|    1|
|          电子社博文视点资讯有限公司|    1|
|上海文艺出版总社，上海锦绣文章出版社|    1|
|                          华中理工大|    1|
|                            湖北科技|    7|
|                       Digireads.com|    2|
|                          林白出版社|    6|
|                    HODDER&STOUGHTON|    1|
|                      北京：三联书店|    1|
|                        上海外语教育|    1|
|                            揚智文化|    1|
+------------------------------------+--

In [12]:
book_pds=book_pds.toPandas()
print(book_pds)

                                  publish  count
0                             SearchPress      3
1     McGraw-HillScience/Engineering/Math      6
2                                    上海教育      5
3                    ScholasticPaperbacks      2
4                                     DAW      1
...                                   ...    ...
4010                                 学苑出版      1
4011                                 青森文化      1
4012                                中国福利会      1
4013                       第1版(1999年1月1日)      1
4014           MacmillanU.K.;NewEdedition      1

[4015 rows x 2 columns]


In [13]:
ax=book_pds.head(n=10).plot(x='publish',y=['count'],kind='bar',title='book count of publish')

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [ ]:
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

spark=SparkSession.builder.appName("rddDataFrame").master("local[*]").getOrCreate()
#读取数据
book=spark.read.text('file:///opt/text/python/pyspark/book.txt')
#转化Rdd
book_rdd=book.rdd.map(lambda x:x[0].split(",")).map(lambda x:Row(id=x[0],name=x[1],rating=x[2],price=x[3],publish=x[4],rul=x[5]))
#创建dataFrame
book_df=spark.createDataFrame(book_rdd)
# book_df.select(book_df['id'],book_df['name'],book_df['rating']-1).show()
#pandas
book_pds=book_df.groupBy('publish').count()
book_pds=book_pds.toPandas()
ax=book_pds.head(n=10).plot(x='publish',y=['count'],kind='bar',title='book count of publish')

# 实验五

In [ ]:
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

spark=SparkSession.builder.appName("rddDataFrame").master("local[*]").getOrCreate()
#读取数据
book=spark.read.text('file:///opt/text/python/pyspark/book.txt')
#rdd
book_rdd=book.rdd.map(lambda x:x[0].split(",")).map(lambda \ x:Row(id=x[0],name=x[1],rating=x[2],price=x[3],publish=x[4],url=x[5]))
#dataframe
book_df=spark.createDataFrame(book_rdd)

